In [ ]:
%matplotlib inline

IGA example
===========

This example demonstrates how to create an IGA input deck.

LS-DYNA version :
ls-dyna\_smp\_s\_R13.0\_365-gf8a97bda2a\_winx64\_ifort190.exe


In [ ]:
import os
import sys


from ansys.dyna.core.pre.dynasolution import DynaSolution
from ansys.dyna.core.pre.dynaiga import (
    DynaIGA,
    IGAPart,
    PartSet,
    IGAFormulation,
    SolidPart,
    HourglassType,
    SolidFormulation,
    RigidwallCylinder,
    Point,
    Curve,
    Direction,
    ContactType,
    ContactCategory,
    OffsetType,
)
from ansys.dyna.core.pre.dynamech import *
from ansys.dyna.core.pre.dynamaterial import MatPiecewiseLinearPlasticity, MatSpotweld
from iga_sample_data import *
from ansys.dyna.core.pre import examples

hostname = "localhost"
if len(sys.argv) > 1:
    hostname = sys.argv[1]

iga_solution = DynaSolution(hostname)
fns = []
path = examples.iga_sample + os.sep
fns.append(path + "maino.k")
fns.append(path + "rkrwelds.key")
fns.append(path + "27parts.key")
iga_solution.open_files(fns)

iga_solution.set_termination(20)
iga_solution.create_database_binary(dt=0.1)

iga = DynaIGA()
iga_solution.add(iga)

iga.set_timestep(timestep_size_for_mass_scaled=-0.0004)

# define material
plastic = MatPiecewiseLinearPlasticity(
    mass_density=7.830e-06, young_modulus=200, yield_stress=1.5, tangent_modulus=0.5
)
swmatlist = []
for mat in materialdata:
    spotweld = MatSpotweld(
        mass_density=mat[0],
        young_modulus=mat[1],
        poisson_ratio=mat[2],
        yield_stress=mat[3],
        plastic_hardening_modulus=mat[4],
        axial_force_resultant_at_failure=mat[5],
        force_resultant_nrs_at_failure=mat[6],
        force_resultant_nrt_at_failure=mat[7],
    )
    swmatlist.append(spotweld)

for id in igaparts:
    part = IGAPart(id)
    part.set_material(plastic)
    part.set_element_formulation(IGAFormulation.REISSNER_MINDLIN_FIBERS_AT_CONTROL_POINTS)
    part.set_thickness(1.0)
    iga.parts.add(part)

for index in range(len(spotwelds)):
    part = SolidPart(spotwelds[index])
    if index != 1:
        part.set_hourglass(type=HourglassType.BELYTSCHKO_BINDEMAN)
    part.set_element_formulation(SolidFormulation.CONSTANT_STRESS_SOLID_ELEMENT)
    part.set_material(swmatlist[index])
    iga.parts.add(part)

cylinder1 = RigidwallCylinder(
    Point(2472.37, -600.000, 1270.98), Point(2472.37, -600.000, 2668.53), 100, 1000
)
iga.add(cylinder1)
cylinder2 = RigidwallCylinder(
    Point(3580.25, -600.000, 1261.37), Point(3580.25, -600.000, 3130.49), 100, 1000
)
iga.add(cylinder2)
cylinder3 = RigidwallCylinder(
    Point(3090.59, -955.35, 1299.42), Point(3090.59, -955.35, 2958.43), 100, 1000
)
cylinder3.set_motion(Curve(x=[0, 100], y=[20, 20]), dir=Direction(0, 1, 0))
iga.add(cylinder3)

# define contact
selfcontact = Contact(type=ContactType.AUTOMATIC)
selfcontact.set_friction_coefficient(static=0.2)
surf1 = ContactSurface(PartSet(igaparts))
selfcontact.set_slave_surface(surf1)
iga.add(selfcontact)

swcontact = Contact(
    type=ContactType.TIED,
    category=ContactCategory.SHELL_EDGE_TO_SURFACE_CONTACT,
    offset=OffsetType.OFFSET,
)
spotweldsolid = ContactSurface(PartSet(spotwelds))
spotweldsurface = ContactSurface(PartSet(igaparts))
swcontact.set_slave_surface(spotweldsolid)
swcontact.set_master_surface(spotweldsurface)
iga.add(swcontact)

iga_solution.save_file()